In [36]:
# import libraries needed for the project

#data handling tools
import pandas as pd
import numpy as np

#data visualization tools
import matplotlib.pyplot as plt
import seaborn as sns

#machine learning tools
# TensorFlow
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import TextVectorization, Embedding, Input, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers import GlobalAvgPool1D

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# other libraries
import random


In [6]:
# import the training and testing data
train_df = pd.read_csv('./Data/train.csv')
test_df = pd.read_csv('./Data/test.csv')

In [7]:
# read the head of the dataset to become familiar with the data
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


since we're not working with a sequential data, we might want to shuffle to prevent the models from learning in some order

In [8]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
# looking at the test set
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
# get 5 random tweets and their classes
random_index = random.randint(0, len(train_df_shuffled)-5)
for row in train_df_shuffled[['text', 'target']][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f'Target: {target}', '(real disaster)' if target > 0 else '(not real disaster)')
  print(f'Text:\n{text}\n')
  print('---')

Target: 1 (real disaster)
Text:
Dr. Bengston on #wildfire management: ÛÏnumbers and size of fires areas affected and costs of fighting them all show upward trend.Û #smem

---
Target: 1 (real disaster)
Text:
#Glimpses: Hyderabad deluged by heavy rainfall | http://t.co/DctV1uJLHc http://t.co/QOx1jNQSAU

---
Target: 0 (not real disaster)
Text:
Worked at a fast food joint. Poured burnt hot oil down the sink. It hit the water in the trap and exploded.  @FallonTonight #WORSTSUMMERJOB

---
Target: 0 (not real disaster)
Text:
@LisaToddSutton The reason I bring this up  bcz he is running 4 Senate. Murphy is nothing but a Republican I am deluged with his junk mail!

---
Target: 1 (real disaster)
Text:
When the answer my friend isn't blowing in the wind-IT'S YELLING! -How to Survive a Dust Storm http://t.co/9NwAJLi9cr http://t.co/tKMOtaeaCo

---


In [11]:
# Let's split the data into training and validation dataset
X_train, X_val, y_train, y_val = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                    train_df_shuffled['target'].to_numpy(),
                                                    test_size=0.1, random_state=42)

In [12]:
# check for sizes of the split
len(X_train), len(X_val), len(y_train), len(y_val)

(6851, 762, 6851, 762)

In [13]:
# visualise the training data
X_train[:11], y_val[0:11]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt',
        'Lava Dragon Breeder! I j

## Text Preprocessing

In [14]:
# check for the average number of tokens in each tweet
avg_token = round(sum([len(i.split()) for i in train_df_shuffled['text']])/len(train_df_shuffled['text']))

In [15]:
# Hyperparameters
max_vocab_length = 10000
max_length = avg_token

# create an instance of the TextVectorization Layer
text_vectorizer = TextVectorization(
    max_tokens = max_vocab_length,
    output_mode = 'int',
    output_sequence_length = max_length
)

In [16]:
# adapt the tokenizer to the training samples
text_vectorizer.adapt(X_train)

In [17]:
# Get an embedding layer to accurately find the relationship among the tokens
embedding = Embedding(
    input_dim = max_vocab_length,
    output_dim = 128,
)

In [18]:
# get the embedding of a random tweet
random_tweet = random.choice(X_train)
print(f'Original tweet:\n{random_tweet}\
        \n\nEmbedded version:')
embedding(text_vectorizer([random_tweet]))

Original tweet:
The Catastrophic Effects of Hiroshima and Nagasaki Atomic Bombings Still Being Felt Today http://t.co/oU1M9chznq        

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.03697232,  0.00887064, -0.00256233, ..., -0.01581059,
          0.00133895, -0.02080971],
        [ 0.02515097,  0.04076488,  0.03841216, ...,  0.02079633,
         -0.04857406, -0.00772405],
        [-0.0110165 , -0.00896317, -0.03603139, ..., -0.00118576,
         -0.04890919, -0.03091152],
        ...,
        [-0.03608364, -0.04327351, -0.03599452, ..., -0.02884655,
          0.02646134, -0.04121586],
        [ 0.01582   , -0.03826817, -0.01559619, ...,  0.01271648,
          0.00563924, -0.00897392],
        [-0.00749024,  0.00532306, -0.03359424, ...,  0.0324089 ,
         -0.02744776,  0.0249236 ]]], dtype=float32)>

## Helper functions

In [19]:
# a list to store all scores so that a dataframe can be created to effectively compare models against the baseline
model_score_list = []

In [32]:
# A function to take a model name, The predicted values and the actual values and return scores
def model_scores(model_name:str, y_preds:list, y_true) -> dict:
  accuracy = accuracy_score(y_true, y_preds)
  precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_preds, average='weighted')
  results = {
      'model_name': model_name,
      'accuracy': accuracy,
      'precision': precision,
      'recall': recall,
      'f1': f1
  }
  model_score_list.append(results)
  return results

# A function to create a tensorboard
def create_tensorboard_callback(experinment_name, dir_name='experinments'):
  log_dir = f"{dir_name}/{experinment_name}/{tf.timestamp}"
  tensorboard_callback = TensorBoard(log_dir=log_dir)
  print(f'Saving TensorBoard log files to: {log_dir}')
  return tensorboard_callback

## Baseline Modelling
For the baseline, we will use a traditional model which in this case is the `naive bayes` classifier

In [21]:
 # create a pipeline for the model
 model_0 = Pipeline([
  ('tfidf', TfidfVectorizer()),
  ('clf', MultinomialNB())
 ])


 # fit the model
 model_0.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [22]:
# evaluate the model
baseline_score = model_0.score(X_val, y_val)
print(f"The baseline Model acheives an accuracy of {baseline_score*100:.2f}%")

The baseline Model acheives an accuracy of 79.27%


In [23]:
# make some predictions with the baseline model
model_0_preds = model_0.predict(X_val)
model_0_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [24]:
# Get the scores for the baseline model
print(model_scores('MultinomialNB', model_0_preds, y_val))

{'model_name': 'MultinomialNB', 'accuracy': 0.7926509186351706, 'precision': 0.8111390004213173, 'recall': 0.7926509186351706, 'f1': 0.7862189758049549}


## Model 1:
A simple neural network with just one output layer

In [37]:
# Get the preprocessing pipeline for the model
inputs = Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = GlobalAvgPool1D()(x)
x = Dense(1, activation='sigmoid')(x)

# build the the model
model_1 = Model(inputs=inputs, outputs=x, name='model_1')

# get the summary of the model
model_1.summary()

Model: "model_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization                   │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [39]:
# compile the model
model_1.compile(loss=BinaryCrossentropy(),
                optimizer=Adam(),
                metrics=['accuracy'])

# fit the model
model_1_history = model_1.fit(X_train, y_train, batch_size=32,
                              epochs=10, validation_data=(X_val, y_val),
                              callbacks=[create_tensorboard_callback('Model_1')],
)

Saving TensorBoard log files to: experinments/Model_1/<function timestamp at 0x7da1d89f89d0>
Epoch 1/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9867 - loss: 0.0302 - val_accuracy: 0.7349 - val_loss: 1.8506
Epoch 2/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9815 - loss: 0.0318 - val_accuracy: 0.7375 - val_loss: 1.8601
Epoch 3/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9849 - loss: 0.0320 - val_accuracy: 0.7375 - val_loss: 1.8564
Epoch 4/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9832 - loss: 0.0316 - val_accuracy: 0.7388 - val_loss: 1.8610
Epoch 5/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9851 - loss: 0.0301 - val_accuracy: 0.7402 - val_loss: 1.8658
Epoch 6/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9846 - loss: 0.0297 - val_accuracy: 0.7402 - val_loss: 1.8730
Epoch 7/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9825 - loss: 0.0348 - val_accuracy: 0.7388 - val_loss: 1.8646
Epoch 8/10


In [43]:
# evaluate the model
model_1.evaluate(X_val, y_val)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7065 - loss: 2.1271


[1.881011724472046, 0.7362204790115356]

In [44]:
# make predictions with the model
model_1_preds = model_1.predict(X_val)
model_1_preds = tf.squeeze(tf.round(model_1_preds))
model_1_preds[:5]==y_val[:5]

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


<tf.Tensor: shape=(5,), dtype=bool, numpy=array([False, False,  True,  True, False])>

In [45]:
# calculate all scores of model 1 and compare to our baseline
print(model_scores('model_1', model_1_preds, y_val))

{'model_name': 'model_1', 'accuracy': 0.7362204724409449, 'precision': 0.7360537314756205, 'recall': 0.7362204724409449, 'f1': 0.736125657413926}


In [47]:
# Create a dataframe of the score list to compare the models
pd.DataFrame(model_score_list).T

,0,1
model_name,MultinomialNB,model_1
accuracy,0.792651,0.73622
precision,0.811139,0.736054
recall,0.792651,0.73622
f1,0.786219,0.736126
